# Yelp API Restaurant Calls By GPS Coordinates

For each coordinate listed in grid_coordinates.csv, return first 1k restaurants found via yelp api and
extract data regarding:

- name
- address & zip
- coordinates
- rating
- review count
- price level
- category
- yelp id

Write data to restaurant_data_coord.csv

In [41]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys1 import yelp_loba_api

### Perform API calls

- Dataframe from grid_coordinates.csv
- Build function that calls and writes restaurant data to new csv, given pair of coordinates
- Run list of coordinates through function

In [42]:
# Create dataframe from csv file
coords_df = pd.read_csv("Resources/grid_coordinates.csv", 
                        names=["Lat", "Lng"])
coords_df.head()

,Lat,Lng
0,38.871657,-123.109843
1,38.871657,-123.017222
2,38.871657,-122.924601
3,38.871657,-122.831980
4,38.871657,-122.739359


In [43]:
# Function returns up to 1000 restaurant listings for input coordinates
def get_restaurants(lat, lng, yelp_loba_api):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % yelp_loba_api}
    restaurant_data = []
    count = 0
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls, radius 4023 meters = 2.5 miles
        params = {"term":"restaurants", "latitude":lat, "longitude":lng, "radius": 4023, 
                  "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Breaks if error occurs with search
        if req.status_code == 400:
            break
        elif req.status_code == 200:
            
            # Convert to json
            response = req.json()
            
            # Log history
            count += 1
            print(f"Now processing query set {count} of max 20")
            
            # Breaks if no further entries in query
            if response["businesses"] == []:
                break
            
            else:
                # Iterate through business results and extract data
                for biz in response["businesses"]:
                    
                    # Logic to replace missing price level data with NaN
                    if "price" not in biz:
                        
                        # Replace both price & missing category data with empty string
                        if biz["categories"] == []:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", "", biz["id"]])
                        else:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", biz["categories"][0]["title"], 
                                                    biz["id"]])

                    # Replace missing category data with empty string
                    elif biz["categories"] == []:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], "", biz["id"]])

                    else:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], 
                                                biz["categories"][0]["title"], biz["id"]])

    # Write to csv
    with open('Resources/restaurant_data_coords_reduced4023radius.csv', 'a', encoding="utf-8") as csvFile:
        writer = csv.writer(csvFile)
        for restaurant in restaurant_data:
            writer.writerow(restaurant)
    csvFile.close()
    
    
    # Returns total count of restaurants in coordinates
    return response["total"]


In [ ]:
# Track # of coordinates processed
count = 0

# List to track total restaurants found in coordinate
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of coordinates
lat_coords = [lat for lat in coords_df["Lat"]]
lng_coords = [lng for lng in coords_df["Lng"]]

for index in range(len(lat_coords)):
                   
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants(lat_coords[index], lng_coords[index] , yelp_loba_api))
    
    # Print log history
    count += 1
    rem = len(lat_coords) - count
    print("-----------------------------------------")
    if count == len(lat_coords):
        print("Full list of coordinates processed!")
    elif count == len(lat_coords) - 1:
        print("Now getting results for final coordinates. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for coordinates complete")
        print(f"Getting results for next coordinates.. there are {rem} coordinates left")
        print("-----------------------------------------")

coords_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 531 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 530 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 529 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 528 coordinates left
-----------------------------------------
Now processing query s

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 497 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 496 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 495 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 494 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
--

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 463 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 462 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 461 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 460 coordinates left
-----------------------------------------
No

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 433 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 432 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 431 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. t

Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 403 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 402 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 401 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 400 coordinates left
-------

...

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 174 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 173 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 172 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 171 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
--

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 142 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
Now processing query set 7 of max 20
Now processing query set 8 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 141 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
Now processing query

Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
Now processing query set 7 of max 20
Now processing query set 8 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 118 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 117 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 116 coordinates left
-----------------------------------------
Now process

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 88 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 87 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 86 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 85 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 55 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 54 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 53 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 52 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
D

In [26]:
# Show coords_df with updated total # of restaurants
coords_df.head()


,Lat,Lng,Total # of Restaurants
0,38.871657,-123.109843,0
1,38.871657,-123.017222,0
2,38.871657,-122.924601,0
3,38.871657,-122.831980,0
4,38.871657,-122.739359,5


In [32]:
coords_df.sort_values(['Total # of Restaurants'], ascending=[False]).head()

,Lat,Lng,Total # of Restaurants
292,37.784307,-122.461496,1200
275,37.856797,-122.276254,1100
293,37.784307,-122.368875,957
294,37.784307,-122.276254,907
412,37.349367,-121.905770,849


In [28]:
# Save restaurants counts to csv
coords_df.to_csv("Resources/total_count_coords_4028radius.csv")

### Create Dataframe

- Read csv file
- Clean data

In [30]:
# Read csv file
restaurants_df = pd.read_csv("Resources/restaurant_data_coords_reduced4023radius.csv", encoding = "ISO-8859-1", 
                            names=["search_ID", "Latitude_search", "Longitude_search", "Name", "Lat_Restaurant", "Lng_Restaurant", "Address", "Zip", 
                                   "Rating", "# of Reviews", "Price Level", "Category", "Yelp ID"])

# Replace NaN entries with blank string
restaurants_df = restaurants_df.fillna('')
restaurants_df

,search_ID,Latitude_search,Longitude_search,Name,Lat_Restaurant,Lng_Restaurant,Address,Zip,Rating,# of Reviews,Price Level,Category,Yelp ID
0,"38.871657, -122.73935900000001",38.871657,-122.739359,Loch Lomond RoadHouse,38.8636,-122.718,10336 Loch Lomond Rd,95461,4.0,48,$$,American (Traditional),lKdiMPiuiKV-ILWD-XgXog
1,"38.871657, -122.73935900000001",38.871657,-122.739359,Infuse Coffee Bar,38.8634,-122.718,10336 Loch Lomond Rd,95461,5.0,4,,Coffee & Tea,_P0y4P7VZtE2pymBmMjJrw
2,"38.871657, -122.73935900000001",38.871657,-122.739359,Brick Oven Pizza,38.825,-122.722,16295 Hwy 175,95426,3.5,36,$$,Pizza,zZEOUAq-9jWHEXG-DSLZsg
3,"38.871657, -122.73935900000001",38.871657,-122.739359,Phat Salads and Wraps,38.3186,-122.3,,94558,4.0,48,$,Food Trucks,i6ZwMjYIbmFZ8EGYz6x5Bw
4,"38.871657, -122.73935900000001",38.871657,-122.739359,808 Island Grill,38.7369,-121.267,,95661,5.0,16,,Hawaiian,fLJLB8y-Bj9AEPVysp0JFQ
5,"38.871657, -122.64673799999998",38.871657,-122.646738,Danny's Road Side Kitchen,38.9116,-122.612,9800 Hwy 53,95457,4.5,65,$$,Barbeque,xNVTY0dPgx_Ay_gN1dMHAQ
6,"38.871657, -122.64673799999998",38.871657,-122.646738,DJ's Pizza,38.91,-122.611,16135 Main St,95457,4.0,41,$$,Pizza,mjsMD4IhP3IZMDYWZ-7CuQ
7,"38.871657, -122.64673799999998",38.871657,-122.646738,Fosters Freeze - Lower Lake,38.9111,-122.612,9860 Highway 53,95457,3.5,32,$,Ice Cream & Frozen Yogurt,fQx_93U_kSL6pNbjaFMWJg
8,"38.871657, -122.64673799999998",38.871657,-122.646738,KFC,38.9111,-122.612,9750 Highway 53,95457,2.0,15,$,Fast Food,D8Eu3doKTl_jN8J8EOb0UQ
9,"38.871657, -122.64673799999998",38.871657,-122.646738,Long John Silver's,38.9121,-122.612,9750 Hwy 53,95457,1.0,3,$$,Fast Food,rb2lspUXj8RKF8JoX7KyQA


In [159]:
### Clean data, how to treat missing address, zip, price level, food trucks/mobile, low review counts
### Groupby on df